In [2]:
%load_ext autoreload
%autoreload 2

In [1]:
from src import loaders,preprocessing,CSP,classifier

import numpy as np
np.set_printoptions(suppress = True)

import matplotlib.pyplot as plt
from sklearn.svm import SVC

from tqdm import tqdm


### Testing FBCSP Classifier with MIBIF Feature Extraction and OvR Classification to see if our implementation matches with Ang et. al (2012)

Starting up the trials run slower for some reason, but they eventually speed up and get to 1.7seconds ish per iteration

In [10]:
#How many times we want to run the experiment per subject
n_times = 10
subject_nos = np.arange(1,10)

fs = 250
frequency_bands = [[f,f+4] for f in range(4,40,4)]

#Computing this only once to save compute time
filter_bank_coefficients = preprocessing.compute_filter_bank_coefficients(fs,frequency_bands)

mean_subject_accuracies = []
standard_errors_acc = []



for subject_no in subject_nos:

    print(f'Running experiments for subject {subject_no}')

    
    trials,labels = loaders.extract_motor_imagery_trials(subject_no)


    filter_bank_trials = preprocessing.filter_bank_trials(trials,fs,filter_bank_coefficients)
    
    evaluation_trials,evaluation_labels = loaders.load_evaluation_trials(subject_no)
    filter_bank_eval = preprocessing.filter_bank_trials(evaluation_trials,fs,filter_bank_coefficients)

    cur_experiment_evaluation_accuracies = []

    for i in tqdm(range(n_times)):
        base_classifier = SVC(kernel = 'linear',probability = True, C = 0.01)

        classifiers,spatial_filters,mibif_indices = classifier.train_classifier(
                                            filter_bank_trials,
                                            labels,
                                            base_classifier = base_classifier,
                                            mode = 'MIBIF')
        
        evaluation_predictions = classifier.make_predictions(
                                    filter_bank_eval,
                                    classifiers,
                                    spatial_filters,
                                    mibif_indices = mibif_indices,
                                    mode = 'MIBIF')
        
        evaluation_accuracy = np.sum(evaluation_predictions == evaluation_labels) / len(evaluation_labels)
        cur_experiment_evaluation_accuracies.append(evaluation_accuracy)

    cur_subject_acc_se = np.std(cur_experiment_evaluation_accuracies) / np.sqrt(n_times)
    standard_errors_acc.append(cur_subject_acc_se)
    
    mean_eval_accuracy = np.mean(cur_experiment_evaluation_accuracies)
    mean_subject_accuracies.append(mean_eval_accuracy)

    
    print('\n')

Running experiments for subject 1


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:15<00:00,  1.54s/it]




Running experiments for subject 2


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:16<00:00,  1.62s/it]




Running experiments for subject 3


...

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:17<00:00,  1.71s/it]




Running experiments for subject 8


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:16<00:00,  1.70s/it]




Running experiments for subject 9


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:14<00:00,  1.46s/it]

In [12]:
subject_accuracy_dict = dict(zip([f'Subject {subject_no}' for subject_no in subject_nos],mean_subject_accuracies))
subject_accuracy_dict

{'Subject 1': 0.6336805555555556,
 'Subject 2': 0.42777777777777776,
 'Subject 3': 0.6791666666666666,
 'Subject 4': 0.42847222222222225,
 'Subject 5': 0.40763888888888883,
 'Subject 6': 0.38506944444444446,
 'Subject 7': 0.7177083333333333,
 'Subject 8': 0.8017361111111111,
 'Subject 9': 0.7364583333333332}

In [13]:
mean_across_subjects = np.mean(mean_subject_accuracies)
mean_across_subjects

0.5797453703703703

In Ang. et al. the mean accuracy across subjects for the evaluation datasets on dataset 2a was 0.569.

Subtle variations probobably occur as we did not do hyper-parameter tuning per SVC model for each subject, and we did not use their specialized Naive Baye's model in their original paper (Support Vector Classifier works just as good too). To compare the subject wise accuracies as well, consult table 2 in Ang et al. (2012) in their paper titled "Filter bank common spatial pattern algorithmon BCIcompetition IV Datasets2a and 2b". Either way, our implementation matches up with their results rather well

i

In [14]:
subject_se_acc_dict = dict(zip([f'Subject {subject_no}' for subject_no in subject_nos],standard_errors_acc))
subject_se_acc_dict

{'Subject 1': 0.0010123180372995277,
 'Subject 2': 0.0008216777476527275,
 'Subject 3': 0.0007283394778959359,
 'Subject 4': 0.0010063456073742707,
 'Subject 5': 0.001568276359810584,
 'Subject 6': 0.002109214242845477,
 'Subject 7': 0.0027362344867840937,
 'Subject 8': 0.0012471031247269439,
 'Subject 9': 0.0012471031247269515}